In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Prepare a local tar.gz file and install the environment

In [2]:
!sudo tar -xvf '/content/drive/MyDrive/recommenders-1.0.0.tar.gz'

recommenders-1.0.0/
recommenders-1.0.0/.devcontainer/
recommenders-1.0.0/.devcontainer/Dockerfile
recommenders-1.0.0/.devcontainer/devcontainer.json
recommenders-1.0.0/.github/
recommenders-1.0.0/.github/.codecov.yml
recommenders-1.0.0/.github/CODEOWNERS
recommenders-1.0.0/.github/ISSUE_TEMPLATE.md
recommenders-1.0.0/.github/ISSUE_TEMPLATE/
recommenders-1.0.0/.github/ISSUE_TEMPLATE/bug_report.md
recommenders-1.0.0/.github/ISSUE_TEMPLATE/feature_request.md
recommenders-1.0.0/.github/ISSUE_TEMPLATE/general-ask.md
recommenders-1.0.0/.github/PULL_REQUEST_TEMPLATE.md
recommenders-1.0.0/.github/dependabot.yml
recommenders-1.0.0/.github/workflows/
recommenders-1.0.0/.github/workflows/actions/
recommenders-1.0.0/.github/workflows/actions/merge-cov/
recommenders-1.0.0/.github/workflows/actions/merge-cov/action.yml
recommenders-1.0.0/.github/workflows/actions/run-tests/
recommenders-1.0.0/.github/workflows/actions/run-tests/action.yml
recommenders-1.0.0/.github/workflows/nightly.yml
recommenders

In [3]:
cd /content/recommenders-1.0.0

/content/recommenders-1.0.0


In [4]:
!python setup.py install

running install
running bdist_egg
running egg_info
creating recommenders.egg-info
writing recommenders.egg-info/PKG-INFO
writing dependency_links to recommenders.egg-info/dependency_links.txt
writing requirements to recommenders.egg-info/requires.txt
writing top-level names to recommenders.egg-info/top_level.txt
writing manifest file 'recommenders.egg-info/SOURCES.txt'
reading manifest file 'recommenders.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
adding license file 'LICENSE'
adding license file 'AUTHORS.md'
writing manifest file 'recommenders.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/recommenders
copying recommenders/__init__.py -> build/lib/recommenders
creating build/lib/tests
creating build/lib/tests/integration
copying tests/integration/__init__.py -> build/lib/tests/integration
creating build/lib/tests/smoke
copying tests/smoke/__init__.

In [5]:
!pip install papermill
!pip install scrapbook

     |████████████████████████████████| 156 kB 7.8 MB/s 
     |████████████████████████████████| 129 kB 32.4 MB/s 
     |████████████████████████████████| 843 kB 31.8 MB/s 
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 5.3.5
    Uninstalling jupyter-client-5.3.5:
      Successfully uninstalled jupyter-client-5.3.5


check GPU

In [24]:
!nvidia-smi

Wed Jan  5 06:31:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Data Preprocess

In [6]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os as os

In [7]:
import sys
import os
import logging
import papermill as pm
import scrapbook as sb
from tempfile import TemporaryDirectory
import numpy as np
import tensorflow.compat.v1 as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED
from recommenders.models.deeprec.deeprec_utils import (
    prepare_hparams
)
# from recommenders.datasets.amazon_reviews import download_and_extract, data_preprocessing
from recommenders.datasets.download_utils import maybe_download


from recommenders.models.deeprec.models.sequential.sli_rec import SLI_RECModel as SeqModel
####  to use the other model, use one of the following lines:
# from recommenders.models.deeprec.models.sequential.asvd import A2SVDModel as SeqModel
# from recommenders.models.deeprec.models.sequential.caser import CaserModel as SeqModel
# from recommenders.models.deeprec.models.sequential.gru4rec import GRU4RecModel as SeqModel
# from recommenders.models.deeprec.models.sequential.sum import SUMModel as SeqModel

#from recommenders.models.deeprec.models.sequential.nextitnet import NextItNetModel

from recommenders.models.deeprec.io.sequential_iterator import SequentialIterator
#from recommenders.models.deeprec.io.nextitnet_iterator import NextItNetIterator

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))


System version: 3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]
Tensorflow version: 2.7.0


Model selection

In [8]:
# yaml_file = '/content/recommenders-1.0.0/recommenders/models/deeprec/config/caser.yaml' 
yaml_file = '/content/recommenders-1.0.0/recommenders/models/deeprec/config/sli_rec.yaml'

In [9]:
EPOCHS = 4
BATCH_SIZE = 400
RANDOM_SEED = SEED  # Set None for non-deterministic result


Set wright data path

In [10]:
reviews_file = "/content/drive/MyDrive/reviews_rating_6_dt.txt"
meta_file = "/content/drive/MyDrive/new_meta_rating.txt"
data_path = "/content/drive/MyDrive/"

#### Parameters Setting

In [11]:
train_file = os.path.join(data_path, r'train_data.txt')
valid_file = os.path.join(data_path, r'new_data_valid.txt')
test_file = os.path.join(data_path, r'new_data_test.txt')
user_vocab = os.path.join(data_path, r'user_vocab.pkl')
item_vocab = os.path.join(data_path, r'item_vocab.pkl')
cate_vocab = os.path.join(data_path, r'category_vocab.pkl')
output_file = os.path.join(data_path, r'output.txt')

reviews_name = 'reviews_rating_6_dt.txt'
meta_name = 'new_meta_rating.txt'

train_num_ngs = 4 # number of negative instances with a positive instance for training
valid_num_ngs = 4 # number of negative instances with a positive instance for validation
test_num_ngs = 4 # number of negative instances with a positive instance for testing
sample_rate = 0.8 # sample a small item set for training and testing here for fast example

input_files = [reviews_file, meta_file, train_file, valid_file, test_file, user_vocab, item_vocab, cate_vocab]

In [12]:
import os
import shutil
import pandas as pd
import gzip
import random
import logging
import _pickle as cPickle

from recommenders.utils.constants import SEED
from recommenders.datasets.download_utils import maybe_download


random.seed(SEED)
logger = logging.getLogger()

In [13]:
reviews_file, meta_file

('/content/drive/MyDrive/reviews_rating_6_dt.txt',
 '/content/drive/MyDrive/new_meta_rating.txt')

In [14]:
def _create_instance(reviews_file, meta_file):
    logger.info("start create instances...")
    dirs, _ = os.path.split(reviews_file)
    output_file = os.path.join(dirs, "instance_output")

    f_reviews = open(reviews_file, "r")
    user_dict = {}
    item_list = []
    for line in f_reviews:
        line = line.strip()
        reviews_things = line.split("\t")
        if reviews_things[0] not in user_dict:
            user_dict[reviews_things[0]] = []
        user_dict[reviews_things[0]].append((line, float(reviews_things[-1])))
        item_list.append(reviews_things[1])

    f_meta = open(meta_file, "r")
    meta_dict = {}
    for line in f_meta:
        line = line.strip()
        meta_things = line.split("\t")
        if meta_things[0] not in meta_dict:
            meta_dict[meta_things[0]] = meta_things[1]

    f_output = open(output_file, "w")
    for user_behavior in user_dict:
        sorted_user_behavior = sorted(user_dict[user_behavior], key=lambda x: x[1])
        for line, _ in sorted_user_behavior:
            user_things = line.split("\t")
            asin = user_things[1]
            if asin in meta_dict:
                f_output.write("1" + "\t" + line + "\t" + meta_dict[asin] + "\n")
            else:
                f_output.write("1" + "\t" + line + "\t" + "default_cat" + "\n")

    f_reviews.close()
    f_meta.close()
    f_output.close()
    return output_file

instance_output = _create_instance(reviews_file, meta_file)

In [ ]:
instance_output = '/content/drive/MyDrive/instance_output'

In [15]:
def _create_item2cate(instance_file):
    logger.info("creating item2cate dict")
    global item2cate
    instance_df = pd.read_csv(
        instance_file,
        sep="\t",
        names=["label", "user_id", "item_id", "timestamp", "cate_id"],
    )
    item2cate = instance_df.set_index("item_id")["cate_id"].to_dict()
    
_create_item2cate(instance_output)

In [17]:
# Sample sample_rate proportion of shop_tag sample 
def _get_sampled_data(instance_file, sample_rate=sample_rate):
    logger.info("getting sampled data...")
    global item2cate
    output_file = instance_file + "_" + str(sample_rate)
    columns = ["label", "user_id", "item_id", "timestamp", "cate_id"]
    ns_df = pd.read_csv(instance_file, sep="\t", names=columns)
    items_num = ns_df["item_id"].nunique()
    items_with_popular = list(ns_df["item_id"])
    items_sample, count = set(), 0
    while count < int(items_num * sample_rate):
        random_item = random.choice(items_with_popular)
        if random_item not in items_sample:
            items_sample.add(random_item)
            count += 1
    ns_df_sample = ns_df[ns_df["item_id"].isin(items_sample)]
    ns_df_sample.to_csv((output_file), sep="\t", index=None, header=None)
    return output_file

sampled_instance_file = _get_sampled_data(instance_output)

In [18]:
def _data_processing(input_file):
    logger.info("start data processing...")
    dirs, _ = os.path.split(input_file)
    output_file = os.path.join(dirs, "preprocessed_output")

    f_input = open(input_file, "r")
    f_output = open(output_file, "w")
    user_count = {}
    for line in f_input:
        line = line.strip()
        user = line.split("\t")[1]
        if user not in user_count:
            user_count[user] = 0
        user_count[user] += 1
    f_input.seek(0)
    i = 0
    last_user = None
    for line in f_input:
        line = line.strip()
        user = line.split("\t")[1]
        if user == last_user:
            if i < user_count[user] - 2:
                f_output.write("train" + "\t" + line + "\n")
            elif i < user_count[user] - 1:
                f_output.write("valid" + "\t" + line + "\n")
            else:
                f_output.write("test" + "\t" + line + "\n")
        else:
            last_user = user
            i = 0
            if i < user_count[user] - 2:
                f_output.write("train" + "\t" + line + "\n")
            elif i < user_count[user] - 1:
                f_output.write("valid" + "\t" + line + "\n")
            else:
                f_output.write("test" + "\t" + line + "\n")
        i += 1
    return output_file

preprocessed_output = _data_processing(sampled_instance_file)

In [ ]:
test_file, valid_file, sampled_instance_file, preprocessed_output, train_file, preprocessed_output

('/content/drive/MyDrive/new_data_test.txt',
 '/content/drive/MyDrive/new_data_valid.txt',
 '/content/drive/MyDrive/instance_output_0.6',
 '/content/drive/MyDrive/preprocessed_output',
 '/content/drive/MyDrive/train_data.txt',
 '/content/drive/MyDrive/preprocessed_output')

In [19]:
def _data_generating(input_file, train_file, valid_file, test_file, min_sequence=1):
    """produce train, valid and test file from processed_output file
    Each user's behavior sequence will be unfolded and produce multiple lines in trian file.
    Like, user's behavior sequence: 12345, and this function will write into train file:
    1, 12, 123, 1234, 12345
    """
    f_input = open(input_file, "r")
    f_train = open(train_file, "w")
    f_valid = open(valid_file, "w")
    f_test = open(test_file, "w")
    logger.info("data generating...")
    last_user_id = None
    for line in f_input:
        line_split = line.strip().split("\t")
        tfile = line_split[0]
        label = int(line_split[1])
        user_id = line_split[2]
        movie_id = line_split[3]
        date_time = line_split[4]
        category = line_split[5]

        if tfile == "train":
            fo = f_train
        elif tfile == "valid":
            fo = f_valid
        elif tfile == "test":
            fo = f_test
        if user_id != last_user_id:
            movie_id_list = []
            cate_list = []
            dt_list = []
        else:
            history_clk_num = len(movie_id_list)
            cat_str = ""
            mid_str = ""
            dt_str = ""
            for c1 in cate_list:
                cat_str += c1 + ","
            for mid in movie_id_list:
                mid_str += mid + ","
            for dt_time in dt_list:
                dt_str += dt_time + ","
            if len(cat_str) > 0:
                cat_str = cat_str[:-1]
            if len(mid_str) > 0:
                mid_str = mid_str[:-1]
            if len(dt_str) > 0:
                dt_str = dt_str[:-1]
            if history_clk_num >= min_sequence:
                fo.write(
                    line_split[1]
                    + "\t"
                    + user_id
                    + "\t"
                    + movie_id
                    + "\t"
                    + category
                    + "\t"
                    + date_time
                    + "\t"
                    + mid_str
                    + "\t"
                    + cat_str
                    + "\t"
                    + dt_str
                    + "\n"
                )
        last_user_id = user_id
        if label:
            movie_id_list.append(movie_id)
            cate_list.append(category)
            dt_list.append(date_time)
            
_data_generating(preprocessed_output, train_file, valid_file, test_file)
#Create train, test, valid data 

In [20]:
def _create_vocab(train_file, user_vocab, item_vocab, cate_vocab):

    f_train = open(train_file, "r")

    user_dict = {}
    item_dict = {}
    cat_dict = {}

    logger.info("vocab generating...")
    for line in f_train:
        arr = line.strip("\n").split("\t")
        uid = arr[1]
        mid = arr[2]
        cat = arr[3]
        mid_list = arr[5]
        cat_list = arr[6]

        if uid not in user_dict:
            user_dict[uid] = 0
        user_dict[uid] += 1
        if mid not in item_dict:
            item_dict[mid] = 0
        item_dict[mid] += 1
        if cat not in cat_dict:
            cat_dict[cat] = 0
        cat_dict[cat] += 1
        if len(mid_list) == 0:
            continue
        for m in mid_list.split(","):
            if m not in item_dict:
                item_dict[m] = 0
            item_dict[m] += 1
        for c in cat_list.split(","):
            if c not in cat_dict:
                cat_dict[c] = 0
            cat_dict[c] += 1

    sorted_user_dict = sorted(user_dict.items(), key=lambda x: x[1], reverse=True)
    sorted_item_dict = sorted(item_dict.items(), key=lambda x: x[1], reverse=True)
    sorted_cat_dict = sorted(cat_dict.items(), key=lambda x: x[1], reverse=True)

    uid_voc = {}
    index = 0
    for key, value in sorted_user_dict:
        uid_voc[key] = index
        index += 1

    mid_voc = {}
    mid_voc["default_mid"] = 0
    index = 1
    for key, value in sorted_item_dict:
        mid_voc[key] = index
        index += 1

    cat_voc = {}
    cat_voc["default_cat"] = 0
    index = 1
    for key, value in sorted_cat_dict:
        cat_voc[key] = index
        index += 1

    cPickle.dump(uid_voc, open(user_vocab, "wb"))
    cPickle.dump(mid_voc, open(item_vocab, "wb"))
    cPickle.dump(cat_voc, open(cate_vocab, "wb"))


_create_vocab(train_file, user_vocab, item_vocab, cate_vocab)    

In [21]:
columns = ["label", "user_id", "item_id", "timestamp", "cate_id"]
ns_df = pd.read_csv(sampled_instance_file, sep="\t", names=columns)
items_with_popular = list(ns_df["item_id"])

global item2cate


In [ ]:
test_file, valid_file

('/content/drive/MyDrive/new_data_test.txt',
 '/content/drive/MyDrive/new_data_valid.txt')

In [22]:
# test negative sampling
logger.info("start test negative sampling")
with open(test_file, "r") as f:
    test_lines = f.readlines()
write_test = open('/content/drive/MyDrive/test_data_test.txt', "w")
for line in tqdm(test_lines):
    write_test.write(line)
    words = line.strip().split("\t")
    positive_item = words[2]
    count = 0
    neg_items = set()
    while count < test_num_ngs:
        neg_item = random.choice(items_with_popular)
        if neg_item == int(positive_item) or neg_item in neg_items:
            continue
        count += 1
        neg_items.add(neg_item)
        words[0] = "0"
        words[2] = neg_item
        words[2] = str(words[2])            
        words[3] = item2cate[neg_item]
        words[3] = str(words[3])
        write_test.write("\t".join(words) + "\n")


100%|██████████| 487973/487973 [00:13<00:00, 36872.25it/s]


In [23]:
# valid negative sampling
logger.info("start valid negative sampling")
with open(valid_file, "r") as f:
    valid_lines = f.readlines()
write_valid = open('/content/drive/MyDrive/valid_data_valid.txt', "w")
for line in tqdm(valid_lines):
    write_valid.write(line)
    words = line.strip().split("\t")
    positive_item = words[2]
    count = 0
    neg_items = set()
    while count < valid_num_ngs:
        neg_item = random.choice(items_with_popular)
        if neg_item == int(positive_item) or neg_item in neg_items:
            continue
        count += 1
        neg_items.add(neg_item)
        words[0] = "0"
        words[2] = neg_item
        words[2] = str(words[2])
        words[3] = item2cate[neg_item]
        words[3] = str(words[3])
        write_valid.write("\t".join(words) + "\n")


100%|██████████| 479123/479123 [00:14<00:00, 33749.28it/s]


In [24]:
test_file = '/content/drive/MyDrive/test_data_test.txt'
valid_file = '/content/drive/MyDrive/valid_data_valid.txt'

### Train model

In [25]:
train_num_ngs = 4 # number of negative instances with a positive instance for training
valid_num_ngs = 4 # number of negative instances with a positive instance for validation
test_num_ngs = 4 # number of negative instances with a positive instance for testing
sample_rate = 0.8 # sample a small item set for training and testing here for fast example

input_files = [reviews_file, meta_file, train_file, valid_file, test_file, user_vocab, item_vocab, cate_vocab]

In [26]:
### NOTE:  
### remember to use `_create_vocab(train_file, user_vocab, item_vocab, cate_vocab)` to generate the user_vocab, item_vocab and cate_vocab files, if you are using your own dataset rather than using our demo Amazon dataset.
hparams = prepare_hparams(yaml_file, 
                          embed_l2=0., 
                          layer_l2=0., 
                          learning_rate=0.001,  # set to 0.01 if batch normalization is disable
                          epochs=EPOCHS,
                          batch_size=BATCH_SIZE,
                          show_step=20,
                          MODEL_DIR=os.path.join(data_path, "model/"),
                          SUMMARIES_DIR=os.path.join(data_path, "summary/"),
                          user_vocab=user_vocab,
                          item_vocab=item_vocab,
                          cate_vocab=cate_vocab,
                          need_sample=True,
                          train_num_ngs=train_num_ngs, # provides the number of negative instances for each positive instance for loss computation.
            )

In [27]:
input_creator = SequentialIterator

In [28]:
model = SeqModel(hparams, input_creator, seed=RANDOM_SEED)

## sometimes we don't want to train a model from scratch
## then we can load a pre-trained model like this: 
#model.load_model(r'your_model_path')

/content/recommenders-1.0.0/recommenders/models/deeprec/models/base_model.py:705: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  training=self.is_train_stage,
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/normalization.py:455: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs, training=training)


In [ ]:
# test_num_ngs is the number of negative lines after each positive line in your test_file
print(model.run_eval(test_file, num_ngs=test_num_ngs)) 

In [ ]:
with Timer() as train_time:
    model = model.fit(train_file, valid_file, valid_num_ngs=valid_num_ngs) 

# valid_num_ngs is the number of negative lines after each positive line in your valid_file 
# we will evaluate the performance of model on valid_file every epoch
print('Time cost for training is {0:.2f} mins'.format(train_time.interval/60.0))
#dt = 8, sample rate = 0.7 

step 20 , total_loss: 1.5980, data_loss: 1.5980
step 40 , total_loss: 1.4930, data_loss: 1.4930
step 60 , total_loss: 1.2408, data_loss: 1.2408
step 80 , total_loss: 1.1519, data_loss: 1.1519
step 100 , total_loss: 1.1104, data_loss: 1.1104
step 120 , total_loss: 0.9842, data_loss: 0.9842
step 140 , total_loss: 0.9592, data_loss: 0.9592
step 160 , total_loss: 0.9946, data_loss: 0.9946
step 180 , total_loss: 0.9081, data_loss: 0.9081
step 200 , total_loss: 0.8689, data_loss: 0.8689
step 220 , total_loss: 0.8566, data_loss: 0.8566
step 240 , total_loss: 0.8842, data_loss: 0.8842
step 260 , total_loss: 0.8513, data_loss: 0.8513
step 280 , total_loss: 0.8662, data_loss: 0.8662
step 300 , total_loss: 0.8985, data_loss: 0.8985
step 320 , total_loss: 0.8977, data_loss: 0.8977


### Select dt < 24 Data

In [10]:
chunksize = 10e5
fields = ['dt', 'chid', 'shop_tag']
chunkdata = pd.read_csv('/content/drive/MyDrive/E-Sun 2021/Cleaned_Shop_tag.csv',usecols=fields ,chunksize=chunksize)
chunk_list = []  # append each chunk df here 

# Each chunk is in df format
for chunk in chunkdata:  
    # perform data filtering 
    # chunk_filter = chunk_preprocessing(chunk)
    
    # Once the data filtering is done, append the chunk to list
    chunk_list.append(chunk)
    
# concat the list into dataframe 
df_concat = pd.concat(chunk_list)

In [11]:
# Avoid using pd.concat in a for loop 
dt = 8
dt_df_concat = []
for ID, df in tqdm(df_concat.groupby('chid')):
    single_customer_dt_sequence = np.unique(df.dt)
    df = df[df.dt.isin(np.sort(single_customer_dt_sequence)[:dt])] 
    dt_df_concat.append(df)
dt_df_concat = pd.concat(dt_df_concat, axis=0)

100%|██████████| 498040/498040 [06:03<00:00, 1371.46it/s]


In [12]:
len(dt_df_concat.chid.unique())

498040

In [ ]:
dt_df_concat.to_csv("C:\\Users\\User\\Documents\\_Keep_12_dt_data.csv")

In [ ]:
shop_tag_list = [2, 6, 10, 12, 13, 15, 18, 19, 21, 22, 25, 26, 36, 37, 39, 48]
sector_shop_tag = df_concat.groupby('shop_tag')

In [ ]:
tag_df_concat = pd.DataFrame(columns=['dt', 'chid', 'shop_tag'])
for tag in shop_tag_list:
    df = sector_shop_tag.get_group((tag))
    tag_df_concat = pd.concat([tag_df_concat, df])
    


In [ ]:
sector_dt = tag_df_concat.groupby('dt')
dt_df_concat = pd.DataFrame(columns=['dt', 'chid', 'shop_tag'])
for dt in range(6,18):
  # dt += 1
  df = sector_dt.get_group(dt)
  dt_df_concat = pd.concat([dt_df_concat, df])
    


### Time Stamp Process

In [ ]:
import time # 引入time
timeString = "2020-09-09 19:00:00" # 時間格式為字串
struct_time = time.strptime(timeString, "%Y-%m-%d %H:%M:%S") # 轉成時間元組
time_stamp = int(time.mktime(struct_time)) # 轉成時間戳
print(time_stamp)

1599678000


In [13]:
import time 
time_String_list = ['2019-01-01 00:00:00', '2019-02-01 00:00:00', '2019-03-01 00:00:00', '2019-04-01 00:00:00',
          '2019-05-01 00:00:00', '2019-06-01 00:00:00', '2019-07-01 00:00:00', '2019-08-01 00:00:00',
          '2019-09-01 00:00:00', '2019-10-01 00:00:00', '2019-11-01 00:00:00', '2019-12-01 00:00:00',
          '2020-01-01 00:00:00', '2020-02-01 00:00:00', '2020-03-01 00:00:00', '2020-04-01 00:00:00',
          '2020-05-01 00:00:00', '2020-06-01 00:00:00', '2020-07-01 00:00:00', '2020-08-01 00:00:00',
          '2020-09-01 00:00:00', '2020-10-01 00:00:00', '2020-11-01 00:00:00', '2020-12-01 00:00:00',]
dt_string_list = list(range(1, 25))

In [14]:
time_stamp_list =[]
for ts in time_String_list:
  struct_time = time.strptime(ts, "%Y-%m-%d %H:%M:%S")
  time_stamp = int(time.mktime(struct_time))
  time_stamp_list.append(time_stamp)


In [ ]:
dt_df_concat['dt'].unique()

array([1546300800, 1551398400, 1554076800, 1556668800, 1559347200,
       1561939200, 1564617600, 1567296000, 1569888000, 1572566400,
       1577836800, 1580515200, 1583020800, 1575158400, 1548979200,
       1601510400, 1585699200, 1598918400, 1606780800, 1593561600,
       1588291200, 1590969600, 1596240000, 1604188800])

In [15]:
dt_df_concat['dt'] = dt_df_concat['dt'].replace(dt_string_list, time_stamp_list)

In [ ]:
dt_df_concat.to_csv("/content/drive/MyDrive/Cleaned_Shop_tag_Select_dt_11_.csv")

### Create review file 

In [16]:
reviews_w = open("/content/drive/MyDrive/reviews_rating_8_dt.txt", "w")
df_concat = dt_df_concat
for i in tqdm(range(len(df_concat))):
    reviews_w.write(
            str(df_concat.iloc[i,1])
            + "\t"
            + str(df_concat.iloc[i,2])
            + "\t"
            + str(df_concat.iloc[i,0])
            + "\n"  )

reviews_w.close()

100%|██████████| 9056749/9056749 [12:46<00:00, 11814.77it/s]


In [17]:
dt_df_concat.dt.unique()

array([1546300800, 1551398400, 1554076800, 1556668800, 1559347200,
       1561939200, 1564617600, 1567296000, 1548979200, 1601510400,
       1585699200, 1577836800, 1598918400, 1575158400, 1580515200,
       1583020800, 1572566400, 1569888000, 1606780800, 1593561600,
       1588291200, 1590969600, 1596240000, 1604188800])

## Load saved model 

In [ ]:
model_best_trained = SeqModel(hparams, input_creator, seed=RANDOM_SEED)
path_best_trained = os.path.join(hparams.MODEL_DIR, "best_model")
print('loading saved model in {0}'.format(path_best_trained))
model_best_trained.load_model(path_best_trained)

/content/recommenders-1.0.0/recommenders/models/deeprec/models/sequential/caser.py:107: UserWarning: `tf.layers.conv1d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv1D` instead.
  name="conv_" + str(shape),
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/convolutional.py:288: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/content/recommenders-1.0.0/recommenders/models/deeprec/models/sequential/caser.py:66: UserWarning: `tf.layers.flatten` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Flatten` instead.
  out_v = tf.compat.v1.layers.flatten(out_v)
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/core.py:523: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/content/recomme

loading saved model in /content/drive/MyDrive/model/best_model


In [ ]:
test_file = "/content/drive/MyDrive/test_data_test.txt"

In [ ]:
model_best_trained.predict(test_file, output_file)

## Transform into Top 3 items

In [ ]:
ns_df = pd.read_csv(output_file, sep="\t", names=['score'], header=None)


In [ ]:
ns_df

,score
0,0.819690
1,0.451304
2,0.266783
3,0.444480
4,0.800881
...,...
1366435,0.282090
1366436,0.601465
1366437,0.656152
1366438,0.349179


In [ ]:
columns = ["label", "user_id", "item_id", "timestamp", "cate_id", "history1", "history2", "history3"]
ns_df2 = pd.read_csv(test_file, sep="\t", names=columns, header=None)


In [ ]:
ns_df2 = ns_df2.iloc[:,:3]
ns_df2

,label,user_id,item_id
0,1,10115966,15
1,0,10115966,48
2,0,10115966,36
3,0,10115966,10
4,1,10484590,15
...,...,...,...
1366435,0,10173550,22
1366436,1,10248632,48
1366437,0,10248632,6
1366438,0,10248632,13


In [ ]:
ns_df = pd.concat([ns_df, ns_df2], axis=1)

In [ ]:
ns_df

,score,label,user_id,item_id
0,0.819690,1,10115966,15
1,0.451304,0,10115966,48
2,0.266783,0,10115966,36
3,0.444480,0,10115966,10
4,0.800881,1,10484590,15
...,...,...,...,...
1366435,0.282090,0,10173550,22
1366436,0.601465,1,10248632,48
1366437,0.656152,0,10248632,6
1366438,0.349179,0,10248632,13


In [ ]:
def get_prediction_output(df):
  sort_df = df.sort_values(by='score').iloc[-3:,:]
  chid = df.user_id.unique()
  top_3, top_2, top_1 = sort_df.item_id
  data = {'chid':chid, 'top1':top_1, 'top2':top_2, 'top3':top_3}
  df = pd.DataFrame(data)
  return df   


In [ ]:
df_output = []
# pd.DataFrame(columns=['chid', 'top1', 'top2', 'top3'])
for dt, df in tqdm(ns_df.groupby('user_id')):
  df_single_user = get_prediction_output(df=df)
  df_output.append(df_single_user)
df_output = pd.concat(df_output, axis=0)

100%|██████████| 341610/341610 [38:45<00:00, 146.89it/s]


In [ ]:
fields = ['chid']
Full_ID_df = pd.read_csv('/content/drive/MyDrive/E-Sun 2021/tbrain_cc_training_48tags_hash_final.csv',usecols=fields)

ID_diff = set(Full_ID_df.chid) - set(df_output.chid)

selection_shop_tag = [2, 10, 15, 36, 37, 39, 48]
probs = [24/120, 24/120, 24/120, 14/120, 24/120, 7/120, 3/120]

for ID in tqdm(ID_diff):
  top_1, top_2, top_3 = np.random.choice(selection_shop_tag, size=3, replace=False, p=probs)
  data = {'chid':[ID], 'top1':[top_1], 'top2':[top_2], 'top3':[top_3]}
  df = pd.DataFrame(data, index=None)
  df_output = pd.concat([df_output, df]) 
 

100%|██████████| 158390/158390 [39:28<00:00, 66.87it/s]


In [ ]:
df_output.to_csv("/content/drive/MyDrive/E-Sun 2021/_colab_3_output.csv", index=None, header=True, index_label=None)